<a href="https://colab.research.google.com/github/sajjadrezvani/NeuromatchProject/blob/main/pyddm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See Also References: [pyddm documentation](https://pyddm.readthedocs.io/en/stable/quickstart.html) ❤

In [1]:
#@title Prepare the environment
!pip -q install git+https://github.com/mwshinn/PyDDM


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# @title Load Dataset
from pyddm import Sample
import pandas
df_rt = pandas.read_csv("data.csv")
df_rt

,motion_coherence,reaction_time,correct,prior_std,raw_response_time
0,0.12,1.715611,0,10,263.873527
1,0.12,1.612799,1,20,339.602631
2,0.24,1.058558,0,20,363.798656
3,0.06,1.014409,1,20,583.970689
4,0.06,1.033698,1,10,632.977531
...,...,...,...,...,...
1909,0.06,1.717913,1,40,157.109099
1910,0.24,1.536026,0,80,64.785959
1911,0.12,2.097255,0,20,501.222311
1912,0.06,2.122608,0,80,241.823091


### Normalizing

In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
scaler = MinMaxScaler()
# scaler = StandardScaler()
df_rt['reaction_time'] = scaler.fit_transform(np.array(df_rt['reaction_time']).reshape(-1, 1))


In [5]:
human_perceptual_sample = Sample.from_pandas_dataframe(df_rt, rt_column_name="reaction_time", choice_column_name="correct")

# Fitting a DDM using PyDDM
First, we want to let the drift rate vary with the coherence. To do so, we must subclass Drift. Each subclass must contain a name (a short description of how drift varies), required parameters (a list of the parameters that must be passed when we initialize our subclass, i.e. parameters which are passed to the constructor), and required conditions (a list of conditions that must be present in any data when we fit data to the model). We can easily define a model that fits our needs:

In [6]:
import pyddm as ddm
class DriftCoherence(ddm.models.Drift):
    name = "Drift depends linearly on coherence"
    required_parameters = ["driftcoh"] # <-- Parameters we want to include in the model
    required_conditions = ["motion_coherence"] # <-- Task parameters ("conditions"). Should be the same name as in the sample.

    # We must always define the get_drift function, which is used to compute the instantaneous value of drift.
    def get_drift(self, conditions, **kwargs):
        return self.driftcoh * conditions['motion_coherence']

In [14]:
from pyddm import Model, Fittable
from pyddm.functions import fit_adjust_model, display_model
from pyddm.models import NoiseConstant, BoundConstant, OverlayChain, OverlayNonDecision, OverlayPoissonMixture
model_rs = Model(name='Roitman data, drift varies with coherence',
                 drift=DriftCoherence(driftcoh=Fittable(minval=0.06, maxval=0.24)),
                 noise=NoiseConstant(noise=1),
                 bound=BoundConstant(B=Fittable(minval=.02, maxval=1.5)),
                 # Since we can only have one overlay, we use
                 # OverlayChain to string together multiple overlays.
                 # They are applied sequentially in order.  OverlayNonDecision
                 # implements a non-decision time by shifting the
                 # resulting distribution of response times by
                 # `nondectime` seconds.
                 overlay=OverlayChain(overlays=[OverlayNonDecision(nondectime=Fittable(minval=0, maxval=1)),
                                                OverlayPoissonMixture(pmixturecoef=.06,
                                                                      rate=1)]),
                 dx=.001, dt=.0001, T_dur=2)

# Fitting this will also be fast because PyDDM can automatically
# determine that DriftCoherence will allow an analytical solution.
fit_model_rs = fit_adjust_model(sample=human_perceptual_sample, model=model_rs, verbose=False)

Info: Params [0.06       0.5011271  0.05562974] gave 524.4469804466706
Info:pyddm:Params [0.06       0.5011271  0.05562974] gave 524.4469804466706


Now let's view the fitted model

In [15]:
import pyddm.plot
pyddm.plot.model_gui_jupyter(model=fit_model_rs, sample=human_perceptual_sample)

Output()

[See Result Explanation](https://github.com/sajjadrezvani/NeuromatchProject/blob/main/DDM-result-explanation.md)

In [9]:
display_model(fit_model_rs)

Model Roitman data, drift varies with coherence information:
Choices: 'correct' (upper boundary), 'error' (lower boundary)
Drift component DriftCoherence:
    Drift depends linearly on coherence
    Fitted parameters:
    - driftcoh: 0.060000 (WARNING: AT BOUNDARY)
Noise component NoiseConstant:
    constant
    Fixed parameters:
    - noise: 1.000000
Bound component BoundConstant:
    constant
    Fitted parameters:
    - B: 0.502149
IC component ICPointSourceCenter:
    point_source_center
    (No parameters)
Overlay component OverlayChain:
    Overlay component OverlayNonDecision:
        Add a non-decision by shifting the histogram
        Fitted parameters:
        - nondectime: 0.055939
    Overlay component OverlayPoissonMixture:
        Poisson distribution mixture model (lapse rate)
        Fixed parameters:
        - pmixturecoef: 0.060000
        - rate: 1.000000
Fit information:
    Loss function: Negative log likelihood
    Loss function value: 524.6782593494507
    Fittin